# Print statements

The simplest way to debug code is to plonk `print` statements in the code. Let's take a common example in which we perform some simple array operations, here multiplying an array and then summing it with another array:

In [1]:
import numpy as np


def array_operations(in_arr_one, in_arr_two):
    out_arr = in_arr_one*1.5
    out_arr = out_arr + in_arr_two
    return out_arr


in_vals_one = np.array([3, 2, 5, 16, '7', 8, 9, 22])
in_vals_two = np.array([4, 7, 3, 23, 6, 8, 0])

result = array_operations(in_vals_one, in_vals_two)
result

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U21'), dtype('float64')) -> None

We've got a `UFuncTypeError` here, perhaps not the most illuminating error message we've ever seen. We'd like to know what's going wrong here. The `Traceback` did give us a hint about where the issue occurred though; it happens in the multiplication line of the function we wrote.

To debug the error with print statements, we might re-run the code like this:

In [2]:
def array_operations(in_arr_one, in_arr_two):
    print(f'in_arr_one is {in_arr_one}')
    out_arr = in_arr_one*1.5
    out_arr = out_arr + in_arr_two
    return out_arr


in_vals_one = np.array([3, 2, 5, 16, '7', 8, 9, 22])
in_vals_two = np.array([4, 7, 3, 23, 6, 8, 0])

result = array_operations(in_vals_one, in_vals_two)
result

in_arr_one is ['3' '2' '5' '16' '7' '8' '9' '22']


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U21'), dtype('float64')) -> None

What can we tell from the values of `in_arr_one` that are now being printed? Well, they seem to have quote marks around them and what that means is that they're strings, *not* floating point numbers or integers! Multiplying a string by 1.5 doesn't make sense here, so that's our error. If we did this, we might then trace the origin of that array back to find out where it was defined and see that instead of `np.array([3, 2, 5, 16, 7, 8, 9, 22])` being declared, we have `np.array([3, 2, 5, 16, '7', 8, 9, 22])` instead and `numpy` decides to cast the whole array as a string to ensure consistency.

Let's fix that problem by turning `'7'` into `7` and run it again:

In [3]:
def array_operations(in_arr_one, in_arr_two):
    out_arr = in_arr_one*1.5
    out_arr = out_arr + in_arr_two
    return out_arr


in_vals_one = np.array([3, 2, 5, 16, 7, 8, 9, 22])
in_vals_two = np.array([4, 7, 3, 23, 6, 8, 0])

result = array_operations(in_vals_one, in_vals_two)
result

ValueError: operands could not be broadcast together with shapes (8,) (7,) 

Still not working! But we've moved on to a different error now. We can still use a print statement to debug this one, which seems to be related to the shapes of variables passed into the function:

In [4]:
def array_operations(in_arr_one, in_arr_two):
    print(f'in_arr_one shape is {in_arr_one.shape}')
    out_arr = in_arr_one*1.5
    print(f'intermediate out_arr shape is {out_arr.shape}')
    print(f'in_arr_two shape is {in_arr_two.shape}')
    out_arr = out_arr + in_arr_two
    return out_arr


in_vals_one = np.array([3, 2, 5, 16, 7, 8, 9, 22])
in_vals_two = np.array([4, 7, 3, 23, 6, 8, 0])

result = array_operations(in_vals_one, in_vals_two)
result

in_arr_one shape is (8,)
intermediate out_arr shape is (8,)
in_arr_two shape is (7,)


ValueError: operands could not be broadcast together with shapes (8,) (7,) 

The print statement now tells us the shapes of the arrays as we go through the function. We can see that in the line before the `return` statement the two arrays that are being combined using the `+` operator don't have the same shape, so we're effectively adding two vectors from two differently dimensioned vector spaces and, understandably, we are being called out on our nonsense. To fix this problem, we would have to ensure that the input arrays are the same shape (it looks like we may have just missed a value from `in_vals_two`).


In [5]:
def array_operations(in_arr_one, in_arr_two):
    print(f'in_arr_one shape is {in_arr_one.shape}')
    out_arr = in_arr_one*1.5
    print(f'intermediate out_arr shape is {out_arr.shape}')
    print(f'in_arr_two shape is {in_arr_two.shape}')
    out_arr = out_arr + in_arr_two
    return out_arr


in_vals_one = np.array([3, 2, 5, 16, 7, 8, 9, 22])
in_vals_two = np.array([4, 7, 3, 23, 6, 8, 0, 3])

result = array_operations(in_vals_one, in_vals_two)
result

in_arr_one shape is (8,)
intermediate out_arr shape is (8,)
in_arr_two shape is (8,)


array([ 8.5, 10. , 10.5, 47. , 16.5, 20. , 13.5, 36. ])

Now, lets comment out (or remove) the debugging statements

In [6]:
def array_operations(in_arr_one, in_arr_two):
    # print(f'in_arr_one shape is {in_arr_one.shape}')
    out_arr = in_arr_one*1.5
    # print(f'intermediate out_arr shape is {out_arr.shape}')
    # print(f'in_arr_two shape is {in_arr_two.shape}')
    out_arr = out_arr + in_arr_two
    return out_arr


in_vals_one = np.array([3, 2, 5, 16, 7, 8, 9, 22])
in_vals_two = np.array([4, 7, 3, 23, 6, 8, 0, 3])

result = array_operations(in_vals_one, in_vals_two)
result

array([ 8.5, 10. , 10.5, 47. , 16.5, 20. , 13.5, 36. ])

`print` statements are great for a quick bit of debugging and you are likely to want to use them more frequently than any other debugging tool. However, for complex, nested code debugging, they aren't always very efficient and you will sometimes feel like you are playing battleships in continually refining where they should go until you have pinpointed the actual problem, so they're far from perfect. Fortunately, there are other tools in the debugging toolbox...